In [ ]:
import os
from dotenv import load_dotenv
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
load_dotenv()

AZURE_OPENAI_KEY = os.getenv("api_key")
AZURE_DEPLOYMENT_NAME = os.getenv("model-name")
API_VERSION = os.getenv("api-version")
AZURE_OPENAI_ENDPOINT = os.getenv("azure_endpoint")

database = {
    "products": {
        "wireless headphones": {"price": 999, "stock": 25, "features": "Noise cancellation, 20hr battery"},
        "smartwatch": {"price": 1999, "stock": 1, "features": "Heart rate monitor, GPS"},
    },
    "orders": {
        "45678": {"product": "wireless headphones", "status": "Out for delivery"},
        "78901": {"product": "smartwatch", "status": "Delivered"},
    },
    "complaints": [],
}

# Create Azure Model Client
model_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_KEY,
    api_version=API_VERSION,
    deployment_name=AZURE_DEPLOYMENT_NAME ,  # Your deployment name
    model =AZURE_DEPLOYMENT_NAME 
)
# Simulated database

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console



# Product Enquiry Tool
def product_enquiry(product_name: str) -> str:
    product = database["products"].get(product_name.lower())
    if product:
        return f"{product_name.title()} costs ₹{product['price']}, features: {product['features']}, stock: {product['stock']}"
    else:
        return "Product not found."


def place_order(product_name: str) -> str:
    product = database["products"].get(product_name.lower())
    if product and product["stock"] > 0:
        order_id = str(len(database["orders"]) + 10000)
        print(f"--------before----------{database}")
        database["orders"][order_id] = {"product": product_name, "status": "Processing"}
        product["stock"] -= 1
        print(f"--------after----------{database}")
        return f"Order placed! Your order ID is {database}."
    else:
        return "Product out of stock or not found."
    
def order_enquiry(order_id: int) -> str:
    order = database["orders"].get(order_id)
    if order:
        return f"Order {order_id} for {order['product']} is currently '{order['status']}'."
    else:
        return "Order ID not found."

def register_complaint(order_id: int, issue: str) -> str:
    if order_id in database["orders"]:
        database["complaints"].append({"order_id": order_id, "issue": issue})
        return f"Complaint registered for order {order_id}: {issue}"
    else:
        return "Invalid order ID."
planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks. It should break down tasks and delegate them to the appropriate agents.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        ProductEnquiryAgent: Handles questions related to product features, availability, and pricing.
        PlaceOrderAgent: Assists users in placing orders for available products.
        OrderEnquiryAgent: Provides updates on the status of existing orders.
        RegisterComplaintAgent: Registers complaints related to products or orders and initiates escalation if needed.
    
    You only plan and delegate tasks - you do not execute them yourself.
    
    When assigning tasks, use this format:
    1. <agent> : <task>
    
    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)
place_order_agent= AssistantAgent(
    "PlaceOrderAgent",
    description="Assists users in placing orders for available products.",
    model_client=model_client,
    tools=[place_order],
    system_message=(
        "You are the Place Order Agent. Help users place orders for products. Confirm availability and "
        "generate order IDs. Update stock accordingly in the database."
    )
)
product_enquiry_agent = AssistantAgent(
    "ProductEnquiryAgent",
    description= "Handles questions related to product features, availability, and pricing.",
    tools= [product_enquiry],
    model_client=model_client,
    system_message= (
        "You are the Product Enquiry Agent. Answer questions about products using the product database. "
        "Provide clear and concise information about features, price, and stock."
    )
)
order_enquiry_agent = AssistantAgent(
        "OrderEnquiryAgent",
        description= "Provides updates on the status of existing orders.",
        tools= [order_enquiry],
        model_client=model_client,
        system_message=(
            "You are the Order Enquiry Agent. Retrieve and report the status of orders using the order ID. "
            "Be accurate and polite."
        )
)
register_complaint_agent = AssistantAgent(
    "RegisterComplaintAgent",
    description= "Registers complaints related to products or orders and initiates escalation if needed.",
    tools= [register_complaint],
    model_client=model_client,
    system_message= (
        "You are the Complaint Registration Agent. Log user complaints and confirm receipt. "
        "Ensure the user feels heard and inform them of next steps."
    )
)
max_messages_termination = MaxMessageTermination(max_messages=10)
team = SelectorGroupChat(
    [planning_agent, place_order_agent, product_enquiry_agent, order_enquiry_agent, register_complaint_agent],
    model_client=model_client,
    termination_condition=max_messages_termination,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)
task = "status for the order id 45678"
await Console(team.run_stream(task=task))
